In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from analysis import *

This notebook compares sensitivities of individual variables, changing one-at-a-time (OAT). The rest of the variables remain as per baseline.

A positive `delta` indicates that buying would result in a financial gain over renting after 5 years.

## Baseline

In [ ]:
print_baseline()

In [ ]:
vars = [
    ("initialPropertyValue", "£", 2),
    ("propertyValueYearlyIncrease", "%", 1),
    ("mortgageInterestRate", "%", 1),
    ("initialMonthlyRentPrice", "£", 3),
    ("rentPriceYearlyIncrease", "%", 1),
    ("savingsInterestRate", "%", 1),
]


def scatter(var, units, slope=False):
    df = load_csv(var)
    y = df["delta"] if slope == False else np.gradient(df["delta"])
    units_prefix = units if units == "£" else ""
    units_suffix = units if units == "%" else ""
    return go.Scatter(
        x=df[var] * infer_multiplier(units),
        y=y,
        name=get_axis_title(var),
        mode="markers+lines",
        hovertemplate=f"{units_prefix}%{{x}}{units_suffix}, £%{{y}}",
    )

In [ ]:
fig = make_subplots(cols=3)
for name, units, col in vars:
    fig.add_trace(scatter(name, units), row=1, col=col)
fig.add_hline(
    56000,
    line_color="green",
    line_dash="dash",
    line_width=0.5,
    annotation_text="Baseline",
)
fig.add_hline(0, line_color="black", line_width=2)
y_range = [-100_000, 250_000]
fig.update_layout(
    xaxis1={"title_text": "Value (%)"},
    xaxis2={"title_text": get_axis_title("initialPropertyValue")},
    xaxis3={"title_text": get_axis_title("initialMonthlyRentPrice")},
    yaxis1={"range": y_range, "title_text": "Delta (£)"},
    yaxis2={"range": y_range, "title_text": "Delta (£)"},
    yaxis3={"range": y_range, "title_text": "Delta (£)"},
)
fig.show()

In [ ]:
fig = make_subplots(cols=3)
for name, units, col in vars:
    fig.add_trace(scatter(name, units, slope=True), row=1, col=col)
y_range = [-20_000, 20_000]
fig.update_layout(
    xaxis1={"title_text": "Value (%)"},
    xaxis2={"title_text": get_axis_title("initialPropertyValue")},
    xaxis3={"title_text": get_axis_title("initialMonthlyRentPrice")},
    yaxis1={"range": y_range, "title_text": "Slope (£/%pt)"},
    yaxis2={"range": y_range, "title_text": "Slope (£/£)"},
    yaxis3={"range": y_range, "title_text": "Slope (£/£)"},
)
fig.show()